<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size

        # Parameters
        self.Wxh = nn.Parameter(torch.randn(input_size, hidden_size) * 0.1)
        self.Whh = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.1)
        self.bh  = nn.Parameter(torch.zeros(hidden_size))

        self.Why = nn.Parameter(torch.randn(hidden_size, output_size) * 0.1)
        self.by  = nn.Parameter(torch.zeros(output_size))

    def forward(self, x, h0=None):
        """
        x: (seq_len, batch, input_size)
        """
        seq_len, batch, _ = x.shape
        if h0 is None:
            h = torch.zeros(batch, self.hidden_size)
        else:
            h = h0

        outputs = []
        for t in range(seq_len):
            xt = x[t]

            h = torch.tanh(xt @ self.Wxh + h @ self.Whh + self.bh)
            y = h @ self.Why + self.by

            outputs.append(y)

        return torch.stack(outputs), h


text = "hello world"
chars = sorted(list(set(text)))
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for c,i in stoi.items()}

def encode(s):
    return torch.tensor([stoi[c] for c in s], dtype=torch.long)

def one_hot(idx, vocab_size):
    return F.one_hot(idx, num_classes=vocab_size).float()

vocab_size = len(chars)
hidden_size = 32

model = MyRNN(vocab_size, hidden_size, vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

data = encode(text)

for epoch in range(500):
    optimizer.zero_grad()

    x = one_hot(data[:-1], vocab_size).unsqueeze(1)  # (seq, batch=1, vocab)
    y_true = data[1:]                                # next char

    y_pred, _ = model(x)
    y_pred = y_pred.squeeze(1)                       # (seq, vocab)

    loss = loss_fn(y_pred, y_true)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"epoch {epoch}, loss={loss.item():.4f}")

def generate(model, start="h", length=20):
    idx = stoi[start]
    h = None
    out = start

    for _ in range(length):
        x = one_hot(torch.tensor([idx]), vocab_size).unsqueeze(1)
        y, h = model(x, h)

        probs = F.softmax(y[-1], dim=0)
        idx = torch.multinomial(probs, 1).item()
        out += itos[idx]

    return out

print(generate(model, "h"))

epoch 0, loss=2.1138
epoch 50, loss=0.0050
epoch 100, loss=0.0019
epoch 150, loss=0.0012
epoch 200, loss=0.0008
epoch 250, loss=0.0006
epoch 300, loss=0.0005
epoch 350, loss=0.0004
epoch 400, loss=0.0003
epoch 450, loss=0.0003
hoeelodhhllde oereww 
